In [12]:
import os
import random
import tempfile
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score

In [13]:
np.random.seed(699)

os.environ['PYTHONHASHSEED'] = '0'
random.seed(699)

In [14]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
path='drive/MyDrive/Capstone(Team10)/code'

#file = tf.keras.utils
raw_df = pd.read_csv(path+'/data/rawdata_USA.csv', index_col=0, parse_dates=True)
raw_df.index.name='date'
metadata=pd.read_csv(path+'/data/metadata_final.csv')

raw_df.tail(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,BNBUCT02,BNEMTE02,BNODTE02,BNRMTE02,BSCICP02,BSCURT02,BSEMFT02,BSOBLV02,BSOITE02,BSPRTE02,...,BBBOASPREAD,SP500,WTI,W875RX1,VIXCLS,RECPROUSM156N,DXY,y_nber,y_oecd,y_agg
date,,,,,,,,,,,,,,,,,,,,,
2023-02-01,NaN,NaN,NaN,NaN,-2.6,78.4,-1.488108,-10.0,-9.904271,-6.838155,...,1.55,3970.153442,76.83,14622.7,20.12,0.36,104.870003,0,0.0,0.0
2023-03-01,NaN,NaN,NaN,NaN,-3.7,77.8,-4.394159,-10.0,-11.664475,-4.339667,...,1.77,4109.312445,73.28,14664.3,21.64,0.32,102.510002,0,0.0,0.0
2023-04-01,NaN,NaN,NaN,NaN,-2.9,78.4,-0.707885,-11.0,-10.242475,-4.047127,...,1.73,4169.481401,79.45,14659.0,17.82,0.22,101.669998,0,0.0,0.0
2023-05-01,NaN,NaN,NaN,NaN,-3.1,78.4,2.680256,-13.0,-17.834084,1.060291,...,1.80,4179.825462,71.58,14702.7,17.64,0.62,104.320000,0,0.0,0.0
2023-06-01,NaN,NaN,NaN,NaN,-4.0,NaN,-5.300056,-9.0,-6.879287,-6.892674,...,1.69,4450.381312,68.11,NaN,14.00,NaN,102.910004,0,0.0,0.0


In [15]:
with open(path+'/data/X_data_full.pkl','rb') as f:
  X_data=pickle.load(f)
X_data.tail(5)

,103SPREAD,LCEAPR01,XTIMVA01,PRMNTO01,MABMM301,PRCNTO01,STMNIS01,SLWHTO02,ODMNTO02,XTEXVA01,...,IRSTFR01,IRLTLT01,IR3TCD01,BSPRTE02,LREMTTTT,LRHU24TT,LRUNTTTT,BSCURT02,3MTBILL,10YT
date,,,,,,,,,,,,,,,,,,,,,
2022-08-01,0.27,0.003972,-0.012969,0.001926,-0.002016,-0.011827,0.000000,0.001179,-0.000768,-0.006967,...,0.65,0.00,0.26,-3.838744,0.2,0.1,0.2,0.1,0.40,0.00
2022-09-01,0.39,0.003597,0.006185,0.002410,-0.006239,-0.005621,0.000000,0.000236,0.001401,-0.008610,...,0.23,0.62,0.45,-0.266136,0.0,0.3,-0.2,0.1,0.50,0.62
2022-10-01,0.26,0.003942,0.010324,0.001324,-0.004279,-0.003519,-0.006826,-0.001735,0.009146,-0.024849,...,0.52,0.46,0.64,0.956603,-0.1,-0.2,0.2,0.0,0.59,0.46
2022-11-01,-0.26,0.004639,-0.066192,-0.007479,-0.001555,0.006387,0.006826,-0.009899,-0.024357,-0.016453,...,0.70,-0.09,0.61,-0.381415,-0.1,0.1,-0.1,-0.6,0.43,-0.09
2022-12-01,-0.63,0.004263,0.020032,-0.021487,-0.001913,-0.000711,0.013514,-0.003170,0.010401,-0.016181,...,0.32,-0.27,0.05,-4.533289,0.2,0.0,-0.1,-1.8,0.10,-0.27


In [16]:
## reference : https://gmnam.tistory.com/230#:~:text=class%20BlockingTimeSeriesSplit%28%29%3A%20def%20__init__%28self%2C%20n_splits%29%3A%20self.n_splits%20%3D%20n_splits,indices%20%5Bstart%3A%20mid%5D%2C%20indices%20%5Bmid%20%2B%20margin%3A%20stop%5D


from sklearn.model_selection import TimeSeriesSplit
from matplotlib.patches import Patch
import matplotlib.pyplot as plt

cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
plt.style.use('fivethirtyeight')

class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

def plot_cv_indices(cv, X, n_splits, lw=10):

    fig, ax = plt.subplots()
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Formatting
    yticklabels = list(range(n_splits))
    ax.set(yticks=np.arange(n_splits) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+0.1, -.1], xlim=[0, len(X)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)

    ax.legend([Patch(color=cmap_cv(.8)), Patch(color=cmap_cv(.02))],
          ['Testing set', 'Training set'], loc=(1.02, .8))

In [17]:
## make index for train, validation dataset
def make_split(X_data, n_splits, test_year):
  train_idxs=[]
  val_idxs=[]

  tss=BlockingTimeSeriesSplit(n_splits=n_splits)

  for train_idx, val_idx in tss.split(X_data[:-(test_year*12)]):
    train_idxs.append(train_idx)
    val_idxs.append(val_idx)

  return train_idxs, val_idxs

In [18]:
def classification_report_csv(report):
    report_data = []
    lines = report.split('\n')

    for line in lines[2:len(lines)-5]:
        row = {}
        row_data = [val for val in line.split(' ') if val!='']
        row['class'] = round(float(row_data[0]),0)
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        row['accuracy']=float([val for val in lines[-4].split(' ') if val!=''][-2])
        report_data.append(row)

    df = pd.DataFrame(report_data)
    return df

1. Random Forest Modeling

In [19]:
## method=class weighted method, 'balanced', 'None'

def rf_grid_search(path, X_data, y_data, y_type, n_splits, test_year, param_rf):

  X_train=X_data.iloc[:-test_year*12]
  y_train=y_data.iloc[:-test_year*12]
  X_test=X_data.iloc[-test_year*12:]
  y_test=y_data.iloc[-test_year*12:]

  tss=BlockingTimeSeriesSplit(n_splits=n_splits)

  model = GridSearchCV(RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=699),
                     param_rf, cv=tss,
                     verbose=3, n_jobs=-1, scoring=['recall_macro'],  ##'recall_macro'
                     refit='recall_macro'
                     )

  clf = make_pipeline(StandardScaler(), model)



  # if y_type!='y_agg':
  #   neg, pos = np.bincount(y_train)
  #   total = neg + pos

  #   weight_for_0 = (1 / neg) * (total / 2.0)
  #   weight_for_1 = (1 / pos) * (total / 2.0)

  #   class_weight = {0: weight_for_0, 1: weight_for_1}

  # else:
  #   neg, pos1, pos2  = np.bincount(y_train)
  #   total = neg + pos1 + pos2

  #   weight_for_0 = (1 / neg) * (total / 3.0)
  #   weight_for_1 = (1 / pos1) * (total / 3.0)
  #   weight_for_2 = (1 / pos2) * (total / 3.0)

  #   class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

  clf.fit(X_train, y_train)

  y_pred_prob=clf.predict_proba(X_test)
  y_pred = clf.predict(X_test)

  clf_report=classification_report(y_test, y_pred)

  param=model.cv_results_['params']
  mean_test_score=model.cv_results_['mean_test_recall_macro']
  std_test_score=model.cv_results_['std_test_recall_macro']
  rank_test_score=model.cv_results_['rank_test_recall_macro']

  for idx, x in enumerate(param):
      x['model']='RF'
      x['y_type']=y_type
      x['mean_test_recall_macro']=mean_test_score[idx]
      x['std_test_recall_macro']=std_test_score[idx]
      x['rank_test_recall_macro']=rank_test_score[idx]

  df_cvresult=pd.DataFrame(param)

  ################# revise
  with open (path+'/model_final/clf_{}_rf.pkl'.format(y_type), 'wb') as f:
      pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)

In [20]:
param_rfs=[{'n_estimators':[30,50,100],
            'max_features':[0.2, 0.3, 0.5, 0.7, 0.9],
            'class_weight':['balanced'],  #,'balanced_subsample',None
            'warm_start':[True,False]}]  ## you can add more dictionary for other combinations of parameters.
y_types=['y_oecd'] #,'y_agg']
test_year=5
n_splits=3
threshold=50

for param_rf in param_rfs:
  for y_type in y_types:
    y=raw_df[y_type]
    y_data=y[-(threshold*12+6):-6]
    rf_grid_search(path, X_data, y_data, y_type, n_splits, test_year, param_rf)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


In [21]:
y_types=['y_oecd'] #,'y_agg']

dict_rf={}

for y_type in y_types:
################# revise
  with open (path+'/model_final/clf_{}_rf.pkl'.format(y_type), 'rb') as f:
      [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
  ################# revise
      dict_rf['clf_{}_rf.pkl'.format(y_type)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]

for idx, model in enumerate(dict_rf):
    if idx==0:
        df_rf=dict_rf[model][1]
    else:
        df_tmp=dict_rf[model][1]
        df_rf=pd.concat([df_rf, df_tmp])

df_rf[df_rf.rank_test_recall_macro==1]

,class_weight,max_features,n_estimators,warm_start,model,y_type,mean_test_recall_macro,std_test_recall_macro,rank_test_recall_macro
18,balanced,0.7,30,True,RF,y_oecd,0.600931,0.067161,1
19,balanced,0.7,30,False,RF,y_oecd,0.600931,0.067161,1


In [22]:
for idx, model in enumerate(dict_rf.keys()):

    report=dict_rf[model][-1]
    y_pred=dict_rf[model][-3]
    y_test=y_data.iloc[-test_year*12:]

    auc=roc_auc_score(y_test, y_pred)

    if idx==0:
        df_rf_creport=classification_report_csv(report)
        df_rf_creport['model']=model
        df_rf_creport['roc_auc']=auc
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_rf_creport=pd.concat([df_rf_creport, df_tmp])
        df_rf_creport['roc_auc']=auc

df_rf_creport

,class,precision,recall,f1_score,support,accuracy,model,roc_auc
0,0.0,0.98,0.77,0.86,56.0,0.77,clf_y_oecd_rf.pkl,0.758929
1,1.0,0.19,0.75,0.30,4.0,0.77,clf_y_oecd_rf.pkl,0.758929


In [23]:
clf[1].best_estimator_.feature_importances_

array([0.06268589, 0.01145694, 0.02347696, 0.06978245, 0.01716118,
       0.03680433, 0.01456736, 0.01567049, 0.02130229, 0.01451954,
       0.01573778, 0.01825744, 0.02382039, 0.01970842, 0.0113888 ,
       0.02412466, 0.02053276, 0.07957944, 0.01913329, 0.0158313 ,
       0.01798132, 0.01199779, 0.02129231, 0.01196119, 0.01330385,
       0.03366717, 0.011506  , 0.0354252 , 0.01351452, 0.01330387,
       0.01824216, 0.02537868, 0.01790757, 0.01236188, 0.02670044,
       0.02973761, 0.01520296, 0.02905623, 0.01434024, 0.00510422,
       0.014697  , 0.00594192, 0.04180599, 0.0117705 , 0.01225768])

In [24]:
rf_probs = list(clf[1].best_estimator_.predict_proba(X_data))
rf_probs

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[array([0.6, 0.4]),
 array([0.6, 0.4]),
 array([0.8, 0.2]),
 array([0.5, 0.5]),
 array([0.6, 0.4]),
 array([0.7, 0.3]),
 array([0.43333333, 0.56666667]),
 array([0.53333333, 0.46666667]),
 array([0.63333333, 0.36666667]),
 array([0.4, 0.6]),
 array([0.7, 0.3]),
 array([0.46666667, 0.53333333]),
 array([0.4, 0.6]),
 array([0.53333333, 0.46666667]),
 array([0.63333333, 0.36666667]),
 array([0.56666667, 0.43333333]),
 array([0.6, 0.4]),
 array([0.53333333, 0.46666667]),
 array([0.46666667, 0.53333333]),
 array([0.6, 0.4]),
 array([0.43333333, 0.56666667]),
 array([0.36666667, 0.63333333]),
 array([0.53333333, 0.46666667]),
 array([0.46666667, 0.53333333]),
 array([0.53333333, 0.46666667]),
 array([0.46666667, 0.53333333]),
 array([0.63333333, 0.36666667]),
 array([0.63333333, 0.36666667]),
 array([0.56666667, 0.43333333]),
 array([0.56666667, 0.43333333]),
 array([0.66666667, 0.33333333]),
 array([0.76666667, 0.23333333]),
 array([0.56666667, 0.43333333]),
 array([0.7, 0.3]),
 array([0.73

In [25]:
lab_feat_imp = list(zip(list(X_data.columns), list(clf[1].best_estimator_.feature_importances_)))
rf_feats=[x for (x, y) in sorted(lab_feat_imp, key=lambda x: x[1], reverse=True)][:10]

2. Random Forsest with Select Features

In [26]:
#rf_feats = ['PRMNTO01','W875RX1','PRINTO01','BSCURT02','LFUNTTTT','PRCNTO01','IRSTFR01','LFHUTTTT','103SPREAD','CPGRLE01']
rf_data = X_data[rf_feats]
# rf_data.head()

In [27]:
param_rfs=[{'n_estimators':[30,50,100],
            'max_features':[0.2, 0.3, 0.5, 0.7, 0.9],
            'class_weight':['balanced'],  #,'balanced_subsample',None
            'warm_start':[True,False]}]  ## you can add more dictionary for other combinations of parameters.
y_types=['y_oecd'] #,'y_agg']
test_year=5
n_splits=3
threshold=50

for param_rf in param_rfs:
  for y_type in y_types:
    y=raw_df[y_type]
    y_data=y[-(threshold*12+6):-6]
    rf_grid_search(path, rf_data, y_data, y_type, n_splits, test_year, param_rf)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


In [29]:
y_types=['y_oecd'] #,'y_agg']

dict_rf={}

for y_type in y_types:
################# revise
  with open (path+'/model_final/clf_{}_rf.pkl'.format(y_type), 'rb') as f:
      [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
  ################# revise
      dict_rf['clf_{}_rf.pkl'.format(y_type)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]

for idx, model in enumerate(dict_rf):
    if idx==0:
        df_rf=dict_rf[model][1]
    else:
        df_tmp=dict_rf[model][1]
        df_rf=pd.concat([df_rf, df_tmp])

df_rf[df_rf.rank_test_recall_macro==1]

,class_weight,max_features,n_estimators,warm_start,model,y_type,mean_test_recall_macro,std_test_recall_macro,rank_test_recall_macro
14,balanced,0.5,50,True,RF,y_oecd,0.602449,0.013537,1
15,balanced,0.5,50,False,RF,y_oecd,0.602449,0.013537,1


In [30]:
for idx, model in enumerate(dict_rf.keys()):

    report=dict_rf[model][-1]
    y_pred=dict_rf[model][-3]
    y_test=y_data.iloc[-test_year*12:]

    auc=roc_auc_score(y_test, y_pred)

    if idx==0:
        df_rf_creport=classification_report_csv(report)
        df_rf_creport['model']=model
        df_rf_creport['roc_auc']=auc
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_rf_creport=pd.concat([df_rf_creport, df_tmp])
        df_rf_creport['roc_auc']=auc

df_rf_creport

,class,precision,recall,f1_score,support,accuracy,model,roc_auc
0,0.0,0.95,0.62,0.75,56.0,0.62,clf_y_oecd_rf.pkl,0.5625
1,1.0,0.09,0.50,0.15,4.0,0.62,clf_y_oecd_rf.pkl,0.5625


In [31]:
lab_feat_imp = list(zip(list(rf_data.columns), list(clf[1].best_estimator_.feature_importances_)))
sorted(lab_feat_imp, key=lambda x: x[1], reverse=True)

[('103SPREAD', 0.13791566350338488),
 ('PRINTO01', 0.116578652853973),
 ('PRMNTO01', 0.1164550300234544),
 ('IRSTFR01', 0.11348079108314582),
 ('W875RX1', 0.09591216122570652),
 ('BSCURT02', 0.09363891718492344),
 ('PRCNTO01', 0.08382491341073087),
 ('CPGRLE01', 0.08152968920875239),
 ('IR3TCD01', 0.08077837593809141),
 ('CSCICP02', 0.07988580556783728)]

In [33]:
rf_sel_probs = list(clf[1].best_estimator_.predict_proba(rf_data))
rf_sel_probs

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[array([0.76, 0.24]),
 array([0.78, 0.22]),
 array([0.72, 0.28]),
 array([0.54, 0.46]),
 array([0.64, 0.36]),
 array([0.7, 0.3]),
 array([0.36, 0.64]),
 array([0.74, 0.26]),
 array([0.68, 0.32]),
 array([0.24, 0.76]),
 array([0.5, 0.5]),
 array([0.38, 0.62]),
 array([0.52, 0.48]),
 array([0.32, 0.68]),
 array([0.82, 0.18]),
 array([0.6, 0.4]),
 array([0.74, 0.26]),
 array([0.62, 0.38]),
 array([0.66, 0.34]),
 array([0.48, 0.52]),
 array([0.26, 0.74]),
 array([0.32, 0.68]),
 array([0.26, 0.74]),
 array([0.24, 0.76]),
 array([0.24, 0.76]),
 array([0.32, 0.68]),
 array([0.46, 0.54]),
 array([0.64, 0.36]),
 array([0.62, 0.38]),
 array([0.72, 0.28]),
 array([0.76, 0.24]),
 array([0.8, 0.2]),
 array([0.82, 0.18]),
 array([0.66, 0.34]),
 array([0.46, 0.54]),
 array([0.74, 0.26]),
 array([0.48, 0.52]),
 array([0.72, 0.28]),
 array([0.84, 0.16]),
 array([0.86, 0.14]),
 array([0.84, 0.16]),
 array([0.74, 0.26]),
 array([0.68, 0.32]),
 array([0.66, 0.34]),
 array([0.66, 0.34]),
 array([0.6, 0.4])

3. Gradient Boosting Decision Tree Classifier

In [46]:
def gb_grid_search(path, X_data, y_data, y_type, n_splits, test_year, param_gb):

  X_train=X_data.iloc[:-test_year*12]
  y_train=y_data.iloc[:-test_year*12]
  X_test=X_data.iloc[-test_year*12:]
  y_test=y_data.iloc[-test_year*12:]

  tss=BlockingTimeSeriesSplit(n_splits=n_splits)

  model = GridSearchCV(GradientBoostingClassifier(random_state=699),
                     param_gb, cv=tss,
                     verbose=3, n_jobs=-1, scoring=['recall_macro'],  #recall_macro
                     refit='recall_macro'
                     )

  clf = make_pipeline(StandardScaler(), model)

  clf.fit(X_train, y_train)

  y_pred_prob=clf.predict_proba(X_test)
  y_pred = clf.predict(X_test)

  clf_report=classification_report(y_test, y_pred)

  param=model.cv_results_['params']
  mean_test_score=model.cv_results_['mean_test_recall_macro']
  std_test_score=model.cv_results_['std_test_recall_macro']
  rank_test_score=model.cv_results_['rank_test_recall_macro']

  for idx, x in enumerate(param):
      x['model']='GB'
      x['y_type']=y_type
      x['mean_test_recall_macro']=mean_test_score[idx]
      x['std_test_recall_macro']=std_test_score[idx]
      x['rank_test_recall_macro']=rank_test_score[idx]

  df_cvresult=pd.DataFrame(param)

  with open (path+'/model_final/clf_{}_gb.pkl'.format(y_type), 'wb') as f:
      pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)

In [47]:
param_gbs=[{'n_estimators':[30,100,300],
            'max_depth':[3,5,None],
            'learning_rate':[0.2, 0.5, 0.8],
            'max_features':[0.2, 0.3, 0.5],
            'warm_start':[True,False]}]
y_types=['y_oecd'] #'y_agg','y_nber']
test_year=5
n_splits=3
threshold=50

for param_gb in param_gbs:
  for y_type in y_types:
    y=raw_df[y_type]
    y_data=y[-(threshold*12+6):-6]
    gb_grid_search(path, X_data, y_data, y_type, n_splits, test_year, param_gb)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


In [48]:
y_types=['y_oecd'] #,'y_agg'

dict_gb={}

for y_type in y_types:
################# revise
  with open (path+'/model_final/clf_{}_gb.pkl'.format(y_type), 'rb') as f:
      [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
  ################# revise
      dict_gb['clf_{}_gb.pkl'.format(y_type)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]

for idx, model in enumerate(dict_gb):
    if idx==0:
        df_gb=dict_gb[model][1]
    else:
        df_tmp=dict_gb[model][1]
        df_gb=pd.concat([df_gb, df_tmp])

df_gb[df_gb.rank_test_recall_macro==1]

,learning_rate,max_depth,max_features,n_estimators,warm_start,model,y_type,mean_test_recall_macro,std_test_recall_macro,rank_test_recall_macro
24,0.2,5.0,0.3,30,True,GB,y_oecd,0.611684,0.068209,1
25,0.2,5.0,0.3,30,False,GB,y_oecd,0.611684,0.068209,1


In [49]:
for idx, model in enumerate(dict_gb.keys()):

    report=dict_gb[model][-1]
    y_pred=dict_gb[model][-3]
    y_test=y_data.iloc[-test_year*12:]

    auc=roc_auc_score(y_test, y_pred)

    if idx==0:
        df_gb_creport=classification_report_csv(report)
        df_gb_creport['model']=model
        df_gb_creport['roc_auc']=auc
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_gb_creport=pd.concat([df_gb_creport, df_tmp])
        df_gb_creport['roc_auc']=auc

df_gb_creport

,class,precision,recall,f1_score,support,accuracy,model,roc_auc
0,0.0,0.95,0.71,0.82,56.0,0.7,clf_y_oecd_gb.pkl,0.607143
1,1.0,0.11,0.50,0.18,4.0,0.7,clf_y_oecd_gb.pkl,0.607143


In [50]:
clf[1].best_estimator_.feature_importances_

array([0.09384004, 0.01362733, 0.02217706, 0.06365543, 0.02368519,
       0.04764423, 0.01398358, 0.01486626, 0.01024027, 0.00820174,
       0.01175737, 0.00204379, 0.01374116, 0.01182846, 0.00664581,
       0.00938303, 0.0157731 , 0.08637923, 0.02446091, 0.01427255,
       0.02001129, 0.01455674, 0.00906295, 0.0270911 , 0.01504412,
       0.03757365, 0.01655942, 0.0516709 , 0.0145588 , 0.01151836,
       0.0207085 , 0.02464879, 0.00945427, 0.00648185, 0.02026156,
       0.04080136, 0.01131257, 0.03144884, 0.01484051, 0.00515969,
       0.00957398, 0.0005869 , 0.02433756, 0.03642836, 0.01810138])

In [51]:
lab_feat_imp = list(zip(list(X_data.columns), list(clf[1].best_estimator_.feature_importances_)))
gb_feats=[x for (x, y) in sorted(lab_feat_imp, key=lambda x: x[1], reverse=True)][:10]

In [52]:
gb_probs = list(clf[1].best_estimator_.predict_proba(X_data))
gb_probs

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


[array([0.90448345, 0.09551655]),
 array([0.82860184, 0.17139816]),
 array([0.91140107, 0.08859893]),
 array([0.70929727, 0.29070273]),
 array([0.863134, 0.136866]),
 array([0.89407385, 0.10592615]),
 array([0.72272462, 0.27727538]),
 array([0.67354751, 0.32645249]),
 array([0.51094148, 0.48905852]),
 array([0.23010658, 0.76989342]),
 array([0.84418806, 0.15581194]),
 array([0.49575753, 0.50424247]),
 array([0.56228909, 0.43771091]),
 array([0.19872218, 0.80127782]),
 array([0.8141281, 0.1858719]),
 array([0.79019736, 0.20980264]),
 array([0.83892854, 0.16107146]),
 array([0.76238738, 0.23761262]),
 array([0.5575073, 0.4424927]),
 array([0.57360346, 0.42639654]),
 array([0.22374684, 0.77625316]),
 array([0.21271226, 0.78728774]),
 array([0.46623722, 0.53376278]),
 array([0.15548759, 0.84451241]),
 array([0.2801798, 0.7198202]),
 array([0.17720448, 0.82279552]),
 array([0.37796333, 0.62203667]),
 array([0.52485381, 0.47514619]),
 array([0.38469576, 0.61530424]),
 array([0.86129833, 0.13

4. Gradient Boosted Tree with Select Features

In [53]:
#gb_feats = ['103SPREAD','BSCURT02','IRSTFR01', 'W875RX1', 'IR3TED01', 'PRINTO01', 'IR3TCD01', 'PIEAEN01', 'ODMNTO02', 'XTIMVA01']
gb_data = X_data[gb_feats]
# gb_data.head()

In [59]:
param_gbs=[{'n_estimators':[30,100,300],
            'max_depth':[3,5,None],
            'learning_rate':[0.2,0.5,0.8],
            'max_features':[0.2, 0.3, 0.5],
            'warm_start':[True,False]}]
y_types=['y_oecd'] #'y_agg','y_nber']
test_year=5
n_splits=3
threshold=50

for param_gb in param_gbs:
  for y_type in y_types:
    y=raw_df[y_type]
    y_data=y[-(threshold*12+6):-6]
    gb_grid_search(path, gb_data, y_data, y_type, n_splits, test_year, param_gb)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


In [60]:
y_types=['y_oecd'] #,'y_agg'

dict_gb={}

for y_type in y_types:
################# revise
  with open (path+'/model_final/clf_{}_gb.pkl'.format(y_type), 'rb') as f:
      [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
  ################# revise
      dict_gb['clf_{}_gb.pkl'.format(y_type)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]

for idx, model in enumerate(dict_gb):
    if idx==0:
        df_gb=dict_gb[model][1]
    else:
        df_tmp=dict_gb[model][1]
        df_gb=pd.concat([df_gb, df_tmp])

df_gb[df_gb.rank_test_recall_macro==1]

,learning_rate,max_depth,max_features,n_estimators,warm_start,model,y_type,mean_test_recall_macro,std_test_recall_macro,rank_test_recall_macro
114,0.8,3.0,0.3,30,True,GB,y_oecd,0.590128,0.020394,1
115,0.8,3.0,0.3,30,False,GB,y_oecd,0.590128,0.020394,1


In [61]:
for idx, model in enumerate(dict_gb.keys()):

    report=dict_gb[model][-1]
    y_pred=dict_gb[model][-3]
    y_test=y_data.iloc[-test_year*12:]

    auc=roc_auc_score(y_test, y_pred)

    if idx==0:
        df_gb_creport=classification_report_csv(report)
        df_gb_creport['model']=model
        df_gb_creport['roc_auc']=auc
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_gb_creport=pd.concat([df_gb_creport, df_tmp])
        df_gb_creport['roc_auc']=auc

df_gb_creport

,class,precision,recall,f1_score,support,accuracy,model,roc_auc
0,0.0,0.97,0.68,0.80,56.0,0.68,clf_y_oecd_gb.pkl,0.714286
1,1.0,0.14,0.75,0.24,4.0,0.68,clf_y_oecd_gb.pkl,0.714286


In [62]:
lab_feat_imp = list(zip(list(gb_data.columns), list(clf[1].best_estimator_.feature_importances_)))
sorted(lab_feat_imp, key=lambda x: x[1], reverse=True)

[('IR3TCD01', 0.24567495661705036),
 ('PRINTO01', 0.17559344241717267),
 ('103SPREAD', 0.13096689304755385),
 ('CPGRLE01', 0.08253360031457796),
 ('PRMNTO01', 0.0779142175762997),
 ('LFHUTTTT', 0.07340465434612098),
 ('W875RX1', 0.06791644163373983),
 ('PRCNTO01', 0.0667899318424535),
 ('IRSTFR01', 0.05341720097502986),
 ('3MTBILL', 0.025788661230001373)]

In [63]:
gb_sel_probs = list(clf[1].best_estimator_.predict_proba(gb_data))
gb_sel_probs

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


[array([0.79473546, 0.20526454]),
 array([0.89236468, 0.10763532]),
 array([0.99184126, 0.00815874]),
 array([0.49380535, 0.50619465]),
 array([0.74162442, 0.25837558]),
 array([0.97199485, 0.02800515]),
 array([0.26192581, 0.73807419]),
 array([0.97953222, 0.02046778]),
 array([0.83987889, 0.16012111]),
 array([0.00148983, 0.99851017]),
 array([0.86465433, 0.13534567]),
 array([0.37888089, 0.62111911]),
 array([0.11099345, 0.88900655]),
 array([0.00325374, 0.99674626]),
 array([0.9948777, 0.0051223]),
 array([0.80053773, 0.19946227]),
 array([0.7616513, 0.2383487]),
 array([0.72045256, 0.27954744]),
 array([0.61505033, 0.38494967]),
 array([0.52471184, 0.47528816]),
 array([0.11113561, 0.88886439]),
 array([0.00364289, 0.99635711]),
 array([0.07526642, 0.92473358]),
 array([0.00559587, 0.99440413]),
 array([0.00666595, 0.99333405]),
 array([0.00343706, 0.99656294]),
 array([0.03141888, 0.96858112]),
 array([0.40237381, 0.59762619]),
 array([0.09011566, 0.90988434]),
 array([0.5563939,

5. Support Vector Machine Classifier

In [ ]:
# def svc_grid_search(path, X_data, y_data, y_type, n_splits, test_year, param_svc):

#   X_train=X_data.iloc[:-test_year*12]
#   y_train=y_data.iloc[:-test_year*12]
#   X_test=X_data.iloc[-test_year*12:]
#   y_test=y_data.iloc[-test_year*12:]


#   # if y_type!='y_agg':
#   #   neg, pos = np.bincount(y_train)
#   #   total = neg + pos

#   #   weight_for_0 = (1 / neg) * (total / 2.0)
#   #   weight_for_1 = (1 / pos) * (total / 2.0)

#   #   class_weight = {0: weight_for_0, 1: weight_for_1}

#   # else:
#   #   neg, pos1, pos2  = np.bincount(y_train)
#   #   total = neg + pos1 + pos2

#   #   weight_for_0 = (1 / neg) * (total / 3.0)
#   #   weight_for_1 = (1 / pos1) * (total / 3.0)
#   #   weight_for_2 = (1 / pos2) * (total / 3.0)

#     # class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

#   tss=BlockingTimeSeriesSplit(n_splits=n_splits)

#   model = GridSearchCV(SVC(random_state=699, probability=True),
#                      param_svc, cv=tss,
#                      verbose=3, n_jobs=-1, scoring=['recall_macro'],
#                      refit='recall_macro'
#                      )

#   clf = make_pipeline(StandardScaler(), model)

#   clf.fit(X_train, y_train)

#   y_pred_prob=clf.predict_proba(X_test)
#   y_pred = clf.predict(X_test)

#   clf_report=classification_report(y_test, y_pred)

#   param=model.cv_results_['params']
#   mean_test_score=model.cv_results_['mean_test_recall_macro']
#   std_test_score=model.cv_results_['std_test_recall_macro']
#   rank_test_score=model.cv_results_['rank_test_recall_macro']

#   for idx, x in enumerate(param):
#       x['model']='SVC'
#       x['y_type']=y_type
#       x['mean_test_recall']=mean_test_score[idx]
#       x['std_test_recall']=std_test_score[idx]
#       x['rank_test_recall']=rank_test_score[idx]

#   df_cvresult=pd.DataFrame(param)

#   ################# revise
#   with open (path+'/model_final/clf_{}_svc.pkl'.format(y_type), 'wb') as f:
#       pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)

In [ ]:
# param_svcs=[{'C':[0.5, 1, 1.5, 2, 5],
#              'kernel':['poly','rbf','sigmoid']}]  ## you can add more dictionary for other combinations of parameters.
# y_types=['y_oecd'] #,'y_oecd']
# test_year=5
# n_splits=3

# for param_svc in param_svcs:
#   for y_type in y_types:
#     svc_grid_search(path, X_data, y_data, y_type, n_splits, test_year, param_svc)

In [ ]:
# y_types=['y_oecd'] #,'y_oecd']

# dict_svc={}

# for y_type in y_types:
# ################# revise
#   with open (path+'/model_final/clf_{}_svc.pkl'.format(y_type), 'rb') as f:
#       [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
#   ################# revise
#       dict_svc['clf_{}_svc.pkl'.format(y_type)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]

# for idx, model in enumerate(dict_svc):
#     if idx==0:
#         df_svc=dict_svc[model][1]
#     else:
#         df_tmp=dict_svc[model][1]
#         df_svc=pd.concat([df_svc, df_tmp])

# df_svc[df_svc.rank_test_recall==1]

In [ ]:
# for idx, model in enumerate(dict_svc.keys()):

#     report=dict_svc[model][-1]

#     if idx==0:
#         df_svc_creport=classification_report_csv(report)
#         df_svc_creport['model']=model
#     else:
#         df_tmp=classification_report_csv(report)
#         df_tmp['model']=model
#         df_svc_creport=pd.concat([df_svc_creport, df_tmp])

# df_svc_creport

In [ ]:
# clf[1].best_estimator_.feature_importances_

In [ ]:
# lab_feat_imp = list(zip(list(X_data.columns), list(clf[1].best_estimator_.feature_importances_)))
# sorted(lab_feat_imp, key=lambda x: x[1], reverse=True)

In [ ]:
# svc_probs = list(clf[1].best_estimator_.predict_proba(X_data))
# svc_probs

In [ ]:
# svc_feats = ['103SPREAD','BSCURT02','IRSTFR01', 'W875RX1', 'IR3TED01', 'PRINTO01', 'IR3TCD01', 'PIEAEN01', 'ODMNTO02', 'XTIMVA01']
# svc_data = X_data[svc_feats]
# # gb_data.head()

6. Support Vector Machine with Select Features

In [ ]:
# param_svcs=[{'C':[0.5, 1, 1.5, 2, 5],
#              'kernel':['poly','rbf','sigmoid']}]  ## you can add more dictionary for other combinations of parameters.

# y_types=['y_oecd'] #'y_agg','y_nber']
# test_year=5
# n_splits=3
# threshold=50

In [ ]:
# for param_svc in param_svcs:
#   for y_type in y_types:
#     y=raw_df[y_type]
#     y_data=y[-(threshold*12+6):-6]
#     svc_grid_search(path, gb_data, y_data, y_type, n_splits, test_year, param_svcs)


In [ ]:
# y_types=['y_oecd'] #,'y_agg'

# dict_svc={}

# for y_type in y_types:
# ################# revise
#   with open (path+'/model_final/clf_{}_gb.pkl'.format(y_type), 'rb') as f:
#       [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
#   ################# revise
#       dict_svc['clf_{}_svc.pkl'.format(y_type)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]

# for idx, model in enumerate(dict_svc):
#     if idx==0:
#         df_svc=dict_svc[model][1]
#     else:
#         df_tmp=dict_svc[model][1]
#         df_svc=pd.concat([df_svc, df_tmp])

# df_svc[df_svc.rank_test_recall==1]


In [ ]:
# for idx, model in enumerate(dict_svc.keys()):

#     report=dict_svc[model][-1]

#     if idx==0:
#         df_svc_creport=classification_report_csv(report)
#         df_svc_creport['model']=model
#     else:
#         df_tmp=classification_report_csv(report)
#         df_tmp['model']=model
#         df_svc_creport=pd.concat([df_svc_creport, df_tmp])


# df_svc_creport

In [ ]:
# lab_feat_imp = list(zip(list(svc_data.columns), list(clf[1].best_estimator_.feature_importances_)))
# sorted(lab_feat_imp, key=lambda x: x[1], reverse=True)

In [ ]:
# svc_sel_probs = list(clf[1].best_estimator_.predict_proba(svc_data))
# svc_sel_probs

Integrating and Storing the results

In [ ]:
dict_list=[df_rf, df_rf_creport, df_gb, df_gb_creport]
id_list=['df_rf', 'df_rf_creport', 'df_gb', 'df_gb_creport']

for i, dict_ in enumerate(dict_list):
################# revise
    with open(path+'/model_final/{}.pkl'.format(id_list[i]), 'wb') as f:
        pickle.dump(dict_, f)